# Thư viện

In [ ]:
import os
import nltk
import numpy as np
import pandas as pd
from collections import Counter
import bisect
import re
import os
import pickle
from nltk.corpus import stopwords
import math
from nltk.stem import WordNetLemmatizer
import gensim
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
ss = SnowballStemmer("english")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NFCorpus

/content/drive/MyDrive/NFCorpus


In [ ]:
!ls

NF_docs  NF_qrel  NF_queries  test.2-1-0.qrel  test.all.queries  test.docs


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
stop_words = nltk.corpus.stopwords.words('english')

# Tiền xử lý

In [ ]:
def preprocess_text(text):
    processed_text = text.lower()
    processed_text = processed_text.replace("’", "'")
    processed_text = processed_text.replace("“", '"')
    processed_text = processed_text.replace("”", '"')
    non_words = re.compile(r"[^A-Za-z']+")
    processed_text = re.sub(non_words, ' ', processed_text)
    return processed_text

In [ ]:
def get_words_from_text(text):
    processed_text = preprocess_text(text)
    filtered_words = [word for word in processed_text.split() if word not in stopwords.words('english')]
    processed_words = []
    for i in range(len(filtered_words)):
        processed_words.append(lemmatizer.lemmatize(filtered_words[i]))
        # processed_words.append(ps.stem(filtered_words[i]))
    return processed_words

In [ ]:
def get_text_from_file(filename):
    with open(filename, encoding='utf-8', mode='r') as f:
        text = f.read()
    return text

In [ ]:
def indexing(docs_path):
    terms = []
    index = []
    norm_list = []
    N = 0

    # Indexing postings
    for doc_file in sorted(os.listdir(docs_path),key=lambda x: int(os.path.splitext(x)[0])):
        filename = os.path.join(docs_path, doc_file)
        N += 1
        text = get_text_from_file(filename)
        words = get_words_from_text(text)
        for word in words:
            if word not in terms:
                terms.append(word)
                index.append([(N, 1)])
            else:
                temp_index = terms.index(word)
                if N not in [i[0] for i in index[temp_index]]:
                    index[temp_index].append((N, 1))
                    continue
                for i in range(len(index[temp_index])):
                    if index[temp_index][i][0] == N:
                        index[temp_index][i] = (N, index[temp_index][i][1] + 1)
                        break

    # Calculate weights
    for i in range(len(index)):
        for j in range(len(index[i])):
            temp = list(index[i][j])
            temp[1] = index[i][j][1] * math.log(N/len(index[i])+1) # Formula: TF * IDF (IDF = log(N / ndoc(t)))
            index[i][j] = tuple(temp)

    # Normalization
    for i in range(N):
        norm = 0
        for j in range(len(index)):
            temp = [item for item in index[j] if item[0] == i+1]
            if len(temp) != 0:
                norm += math.pow(temp[0][1], 2)
        norm_list.append(math.sqrt(norm))
    for i in range(len(index)):
        for j in range(len(index[i])):
            temp = list(index[i][j])
            temp[1] = index[i][j][1] / norm_list[index[i][j][0]-1]
            index[i][j] = tuple(temp)

    return terms, index

In [ ]:
terms, index = indexing(r'./NF_docs')

In [ ]:
print(len(terms))

19917


In [ ]:
print(index[:2])

[[(1, 0.7233587586213442), (2, 0.4109294221350846), (156, 0.30756026615496546), (168, 0.10333254993500658), (211, 0.12478749215457015), (230, 0.07175042715866932), (342, 0.07646633272836052), (391, 0.21854066586261967), (454, 0.06986466371799159), (528, 0.0488871183626622), (731, 0.07699466486799758), (741, 0.16756860328683065), (1163, 0.3618981031715568), (1164, 0.5610795333150008), (1166, 0.5650329266217543), (1167, 0.07699466486799758), (1230, 0.06519209599599779), (1623, 0.0930109541552868), (1745, 0.06677164749124415), (2597, 0.5521508019127087), (2806, 0.1182305020031857), (2808, 0.3994935516642588), (2809, 0.5038351991456685), (2810, 0.5142890852705635)], [(1, 0.25746168633368666), (2, 0.475345583861006), (11, 0.030023120259717356), (63, 0.16033847525289766), (65, 0.03187550302679276), (75, 0.0261294025699638), (122, 0.08425071000547192), (132, 0.03699120962576088), (144, 0.033017801359831664), (146, 0.030395990378806432), (185, 0.2612455820908789), (186, 0.04001412052637377), (

# Truy vấn

In [ ]:
def load_query(path):
    query = []
    for doc_file in sorted(os.listdir(path),key=lambda x: int(os.path.splitext(x)[0])):
        filename = os.path.join(path, doc_file)
        with open(filename, encoding='utf-8', mode='r') as f:
            temp2 = f.readlines()
        listToStr = ' '.join([str(elem) for elem in temp2])
        temp = ""
        for i in listToStr.split():
          content = i.replace('.', '').replace("''", '').replace('\n', '').lower()
          content = gensim.utils.simple_preprocess(content)
          content = [' '+non_stopword for non_stopword in content if non_stopword not in stop_words]
          # content = [ps.stem(word) for word in content]
          content = [lemmatizer.lemmatize(word) for word in content]
          temp +=  " ".join(content)
        query.append(temp)
    return query

In [ ]:
def load_res(res_path):
      goal = []
      for doc_file in sorted(os.listdir(res_path),key=lambda x: int(os.path.splitext(x)[0])):
        filename = os.path.join(res_path, doc_file)
        with open(filename, encoding='utf-8', mode='r') as f:
            temp2 = f.readlines()
        temp3 = []
        a = []
        for x in temp2:
          a = x.strip().split()
          for i in a:
            temp3.append(int(i))
        goal.append(temp3)
      return goal

In [ ]:
def get_search_results(query, terms, index):
    words = get_words_from_text(query)

    # Get postings
    temp_postings = {}
    temp_words = list(set(words))
    for i in range(len(temp_words)):
        if temp_words[i] in terms:
            temp_postings[temp_words[i]] = index[terms.index(temp_words[i])].copy()

    # Calculating TF * IDF (IDF = log(N / ndoc(t)))
    N = []
    for i in range(len(index)):
        N.append(max(index[i],key=lambda item:item[0])[0])
    N = max(N)
    temp = [item for item in words if item in terms]
    weight_list = Counter(temp)
    weight_list = {x: weight_list[x] * math.log(N / len(temp_postings[x])) for x in weight_list}

    # Normalization
    norm = 0
    temp = list(weight_list.values())
    for i in range(len(temp)):
        norm += math.pow(temp[i], 2)
    norm = math.sqrt(norm)
    weight_list = {x: weight_list[x] / norm for x in weight_list}

    for i in temp_postings:
        for j in range(len(temp_postings[i])):
            temp = list(temp_postings[i][j])
            temp[1] *= weight_list[i]
            temp_postings[i][j] = tuple(temp)

    # Get search result(s)
    res = list(temp_postings.values())
    result = {}
    for L in res:
        for key, value in L:
            result[key] = result.get(key, 0) + value
    result = list(result.items())

    return sorted(result, key=lambda tup: tup[1], reverse=True)

In [ ]:
def evaluateMAP(test_query, test_path, terms, index):
    PRECISION = []
    RECALL = []

    query = load_query(test_query)
    goal = load_res(test_path)

    average_precision = []

    for i in range(len(goal)):
        temp4 = get_search_results(query[i], terms, index)
        temp5 = [i[0] for i in temp4] # lấy ra danh sách các document trả về sắp xếp theo độ phù hợp giảm dần

        precision = []
        recall = []
        count = 0
        n = 0
        temp_precision = []

        # tính AP và recall
        for j in range(len(temp5)):
            if temp5[j] in goal[i]:
                count += 1
            n += 1
            if (len(goal[i])==0):
                recall.append(count / (len(goal[i])+1))
            else:
                recall.append(count / len(goal[i]))
            precision.append(count / n)

        # tính AP nội suy
        temp = 0
        for i in range(len(precision)-1, -1, -1):
            if temp >= precision[i]:
                precision[i] = temp
            elif temp < precision[i]:
                temp = precision[i]

        # tính MAP 11 điểm
        precision_list_11 = []
        points = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        for idx, i in enumerate(points):
            for j in range(len(recall)):
                if recall[j] >= i:
                    precision_list_11.append(max(precision[j:]))
                    break
        if len(precision_list_11) < 11:
          precision_list_11 += [0]*(11-len(precision_list_11))

        if len(precision_list_11) != 0:
            average_precision.append(sum(precision_list_11) / 11)
        else:
            average_precision.append(0)
        PRECISION.append(precision[-1])
        RECALL.append(recall[-1])
    if len(average_precision) != 0:
        return sum(average_precision) / len(average_precision), sum(PRECISION) / len(PRECISION), sum(RECALL) / len(RECALL)
    else:
        return 0, sum(PRECISION) / len(PRECISION), sum(RECALL) / len(RECALL)

In [ ]:
queries_path = './NF_queries'
qrel_path = './NF_qrel'

# Kết quả Porter Stemmer

In [ ]:
import time
start_time = time.time()
mAP, precision, recall = evaluateMAP(queries_path, qrel_path , terms, index)
print('mAP = ', mAP )
print('Precision = ', precision )
print('Recall = ', recall )
print("Time: %s seconds" % (time.time() - start_time))

mAP =  0.005441588548039565
Precision =  0.005829274195979898
Recall =  0.2775801554205905
Time: 146.23704934120178 seconds


# Kết quả Lemmatizer


In [ ]:
import time
start_time = time.time()
mAP, precision, recall = evaluateMAP(queries_path, qrel_path , terms, index)
print('mAP = ', mAP )
print('Precision = ', precision )
print('Recall = ', recall )
print("Time: %s seconds" % (time.time() - start_time))

mAP =  0.005587448769977661
Precision =  0.005788083139174165
Recall =  0.2776130969402266
Time: 148.59757709503174 seconds
